<a href="https://colab.research.google.com/github/sahaanirbannew/spacy-custom-NER-bird-names/blob/main/Connecting_to_Dropbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The Process:**


1.   Edit a photograph on Adobe Lightroom, export it to a Dropbox folder.
2.   Program takes image from Dropbox folder, posts it with description. 



In [45]:
!pip3 install dropbox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [67]:
import pathlib
import pandas as pd
import dropbox
from dropbox.exceptions import AuthError
import tweepy 
import pickle 
import csv
import datetime 
from datetime import datetime 
import operator
import time
import requests
import os
import re

log = {}
log["info"] = []
log["error"] = []

def to_log(type_,text):
  if type_ == "i":
    log["info"].append(text)
  if type_ == "e":
    log["error"].append(text)


def connect_to_dropbox(token):
  try:
    TOKEN = token
    dbx = dropbox.Dropbox(TOKEN)
    to_log("i","Dropbox connection made.") 
    return dbx 
  except Exception as e:
    to_log("e","Dropbox connection failed.")  

def list_files_in_folder(dbx, dropbox_folder):
  files_ = []
  try: 
    files = dbx.files_list_folder(dropbox_folder).entries
    if len(files)>0: to_log("i","There are files existing in the folder.") 
    for file in files:
      files_.append([file.name,file.path_lower]) 
    return files_
  except Exception as e:
    to_log("e",str(e))

def fetch_already_uploaded_image_list():
  try:
    file = open("/content/already_uploaded_image_list",'rb')
    bird_list_list = pickle.load(file) 
    return bird_list_list
  except Exception as e: 
    to_log("e",str(e))
    return [["0.jpg","o.jpg"]]

def create_twitter_app_obj():
  consumer_key = "iPaIdR8GRI59yTJMs0Es0dIBN"
  consumer_secret = "pLadg3UaLeK3yKDujRMChRN3p8hUDBOjBsuOBy8j8ERr4zz1vs"
  access_token = "39085479-AabHt6bmFSbClDfUZuHjModYPAxVlOxHeMA79UyVt"
  access_token_secret = "3IqXDISfqg14wzMNNn2AX4KYG9Wfkltt21QxKasE4YNnG"
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_token_secret) 

  try:
    api = tweepy.API(auth) 
    return api
  except Exception as e:
   to_log("e",str(e))
  return 0

def download_files_from_dropbox(old_list,folder_name,dbx):
  files = dbx.files_list_folder(folder_name).entries
  for file in files:
    if file.name in files__:
      file_path=file.path_lower
      file_name=file.name
      dbx.files_download_to_file(download_path="/content/"+file_name,path=file_path)

def dump_already_uploaded_image_list(list_of_files):
  filepath = open("/content/already_uploaded_image_list",'wb') 
  pickle.dump(list_of_files, filepath)                     
  filepath.close() 

def get_context_text(link_to_context, dbx):
  #download the context file. 
  dbx.files_download_to_file("/content/context.txt",link_to_context)

  #read the text
  with open("/content/context.txt") as f:
    lines = f.readlines()
  return lines

def find_all_hashtags(api, search_word):
  new_search = "#" +search_word + " -filter:retweets"  
  hashtags = {}

  for tweet in tweepy.Cursor(api.search,q=new_search,count=100,
                            lang="en",
                            since_id=0, tweet_mode="extended").items(): 

    for tag in tweet.entities["hashtags"]:
      if tag["text"] not in hashtags: 
        hashtags[tag["text"]] = 1
      else: 
        hashtags[tag["text"]] += 1

  return hashtags

def return_top_hashtags(hashtags, k): 
  sorted_d = dict(sorted(hashtags.items(), key=operator.itemgetter(1),reverse=True))
  count = 0
  top_hashtags_str = "" 

  for hashtag in sorted_d:  
    top_hashtags_str = top_hashtags_str + " #" + hashtag 
    count += 1
    if count == k: break 
  return top_hashtags_str

def return_tweet_text(lines, hashtags_str):
  tweet_text = "" 
  for line in lines: 
    tweet_text = tweet_text + line 
  tweet_text = tweet_text + hashtags_str
  return tweet_text

def validate_birdname(birdname):
  birdname = ''.join((x for x in birdname if not x.isdigit())) 
  api_url = "https://bird-name-ner-nlp.herokuapp.com/ner?sent="+birdname
  response = requests.get(api_url).json()  
  if len(response['bird-rule'])>0 or len(response['bird-ner'])>0:
    return birdname
  return ""

def download_image_to_local(dropbox_filepath, filename, dbx):
  dbx.files_download_to_file("/content/"+filename,dropbox_filepath)
  size = os.path.getsize("/content/"+filename)
  if size>4882999: print("Maybe it exceeds the size limit by Dropbox")
  return size,"/content/"+filename

def add_new_entries_to_list(already_uploaded_image_list,to_upload): 
  for index, row in to_upload.iterrows():
    already_uploaded_image_list.loc[len(already_uploaded_image_list.index)] = [row['name'],row['filepath']] 
  return already_uploaded_image_list

def trigger_screen_response():
  with_ = input("Who were you with? (Enter Twitter usernames) ")
  place = input("Where did you photograph the birds? ")
  date = input("When did you photograph the birds? dd.mm.yyyy ")
  return with_, place, date

def post_to_twitter(to_upload,lines, hashtags_str,twitter): 
  for index, row in to_upload.iterrows():
    tweet_text = return_tweet_text(lines, hashtags_str)
    birdname_mit_nummer = row["name"].split(".")[0].lower() 
    birdname = validate_birdname(birdname_mit_nummer) 

    if len(birdname)==0: 
      print(birdname_mit_nummer, " --skipped") 
      continue 

    tweet_text =  birdname + " " + tweet_text 
    filesize, local_path= download_image_to_local(row["filepath"], row["name"], dbx) 
    print(filesize) 
    
    try:
      media = twitter.media_upload(local_path)
      post_result = twitter.update_status(status=tweet_text, media_ids=[media.media_id]) 
    except Exception as e:
      to_log("e",str(e))
      print(birdname, str(e))

    print(local_path)
    print(str(datetime.now())+"Posted - "+local_path)
    os.remove(local_path) #this did not work. I dont know why. 

    time.sleep(10)

In [71]:
# Hard Coded 
token = input("Input Token: ")
if token == "-": token = "DB__sl.BO2pntGFDyA6Js47RwJcGeRw3lT2xx3viN82cveWTrSjSwA4-ql7Q7W95LtPItb0Twf19ZKmuDgum15Z6e2W0DrBctvu7mWjGO0GXcoh5Udqfky0pjYT7EeBSBFrjmHYnhA_enxL"
search_word = "IndiAves" #It is okay to hard code this. 

#Variables
folder_name = "/"+input("Enter folder name: ").strip()  
max_hashtags = 5 

 
lines = ""  
dbx = connect_to_dropbox(token)
already_uploaded_image_list = pd.DataFrame(fetch_already_uploaded_image_list() , columns =['name', 'filepath'])
present_set_of_images = pd.DataFrame(list_files_in_folder(dbx, folder_name) , columns =['name', 'filepath']) 
try:
  link_to_context = present_set_of_images['filepath'].where(present_set_of_images['name'] == "context.txt").iloc[0]
  lines = get_context_text(link_to_context, dbx)  
except Exception as e: 
  print(str(e))
  with_, place, date = trigger_screen_response()
  if len(place)<1 or len(place)<8:
    print("You did not enter required fields. Exiting.") 
  if len(with_)>1: 
    lines = " | place: "+place+" | date: "+date +" | with: "+with_
  else:
    lines = " | place: "+place+" | date: "+date

Input Token: -
Enter folder name: empty
'nan' expected to be a string, got float
Who were you with? (Enter Twitter usernames) @shotsbynavin
Where did you photograph the birds? Rajarhat, Kolkata
When did you photograph the birds? dd.mm.yyyy 07.09.2022


In [72]:
if len(lines)>1:
  twitter = create_twitter_app_obj()
  try_again = True
  while (try_again == True): #at times it takes time.
    try:
      hashtags = find_all_hashtags(twitter,search_word) 
      hashtags_str = return_top_hashtags(hashtags, max_hashtags) 
      try_again = False 
    except Exception as e:
      print(str(e)) 
      try_again = True 
      time.sleep(5)

Twitter error response: status code = 500
Twitter error response: status code = 500
Twitter error response: status code = 503


In [73]:
reponse = dbx.check_and_refresh_access_token()
print(reponse)

None


In [74]:
while 1>0: 
  present_set_of_images = pd.DataFrame(list_files_in_folder(dbx, folder_name) , columns =['name', 'filepath']) 
  to_upload = present_set_of_images[~present_set_of_images.name.isin(already_uploaded_image_list.name)]
  print(to_upload)
  already_uploaded_image_list = add_new_entries_to_list(already_uploaded_image_list,to_upload)
  post_to_twitter(to_upload,lines, hashtags_str)
  time.sleep(600)

                     name                       filepath
0  red munia (male) 5.jpg  /empty/red munia (male) 5.jpg
1  red munia (female).jpg  /empty/red munia (female).jpg
Maybe it exceeds the size limit by Dropbox
4014666
/content/red munia (male) 5.jpg
2022-09-07 10:14:08.526314Posted - /content/red munia (male) 5.jpg
Maybe it exceeds the size limit by Dropbox
3647692
/content/red munia (female).jpg
2022-09-07 10:14:23.503779Posted - /content/red munia (female).jpg
                       name                         filepath
2  scaly breasted munia.png  /empty/scaly breasted munia.png
Maybe it exceeds the size limit by Dropbox
11285598
scaly breasted munia File is too big, must be less than 4883kb.
/content/scaly breasted munia.png
2022-09-07 10:24:35.529363Posted - /content/scaly breasted munia.png


KeyboardInterrupt: ignored